# Projeção do IDHM para Subprefeituras de São Paulo

Projeção do Índice de Desenvolvimento Humano Municipal (IDHM) para 2020

Este script realiza a projeção dos valores do IDHM (geral, renda e educação) das subprefeituras de São Paulo 
para o ano de 2020, com base nos dados disponíveis dos anos de 2000 e 2010.

Duas abordagens são aplicadas:
- Projeção Linear: estima o crescimento com base na diferença entre 2000 e 2010.
- Projeção Exponencial: considera o crescimento percentual médio anual e aplica sobre 20 anos.

Etapas do script:
- Carrega a base de dados (formato .xlsx ou .csv),
- Calcula as projeções para 2020,
- Exporta os resultados para um novo arquivo Excel.

Desenvolvido em Python 3.13 no IDLE e adaptado para uso em Jupyter Notebook.   

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd  # Manipulação de dados
import matplotlib.pyplot as plt  # Visualização de gráficos
import seaborn as sns  # Estilização dos gráficos
import numpy as np  # Operações matemáticas e estatísticas
from sklearn.linear_model import LinearRegression  # Regressão linear

In [ ]:
# Definir o arquivo de entrada
arquivo_entrada = "idhm_subpref_anos.xlsx"  # ou "idhm_subpref_anos.xlsx.csv"

In [ ]:
try:
    # Criar diretório para salvar os resultados, se não existir
    os.makedirs("analises/projecoes_idhm", exist_ok=True)

    # Carregar os dados do IDHM
    df_idhm = pd.read_excel("idhm_2000_2021.xlsx")

    # Listar as colunas de anos para análise
    colunas_idhm = [
        "IDHM_2000", "IDHM_2010", "IDHM_2015", "IDHM_2017",
        "IDHM_2018", "IDHM_2019", "IDHM_2020", "IDHM_2021"
    ]

    # Converter colunas para numérico, tratando erros como NaN
    df_idhm[colunas_idhm] = df_idhm[colunas_idhm].apply(pd.to_numeric, errors="coerce")

    def projetar_idhm(subprefeitura_df, anos_futuros):
        """Realiza regressão linear para projetar o IDHM."""
        X = np.array([int(ano.split("_")[1]) for ano in colunas_idhm]).reshape(-1, 1)
        y = subprefeitura_df[colunas_idhm].values.flatten()

        # Remover valores nulos para o ajuste
        mask = ~np.isnan(y)
        X_clean = X[mask]
        y_clean = y[mask]

        model = LinearRegression()
        model.fit(X_clean, y_clean)

        # Fazer projeções
        X_future = np.array(anos_futuros).reshape(-1, 1)
        previsoes = model.predict(X_future)
        return previsoes

    # Definir os anos futuros para projeção
    anos_futuros = [2025, 2030, 2035]
    resultados = []

    # Loop pelas subprefeituras e projetar IDHM
    for _, row in df_idhm.iterrows():
        subprefeitura = row["Prefeitura Regional"]
        previsoes = projetar_idhm(row.to_frame().T, anos_futuros)
        for ano, valor in zip(anos_futuros, previsoes):
            resultados.append({
                "Prefeitura Regional": subprefeitura,
                "Ano": ano,
                "Projecao_IDHM": valor
            })

    # Salvar resultados
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv("analises/projecoes_idhm/projecoes_idhm.csv", index=False)

    # Gráfico geral das projeções
    plt.figure(figsize=(14, 8))
    for subprefeitura in df_resultados["Prefeitura Regional"].unique():
        sub_df = df_resultados[df_resultados["Prefeitura Regional"] == subprefeitura]
        plt.plot(sub_df["Ano"], sub_df["Projecao_IDHM"], marker="o", label=subprefeitura)

    plt.title("Projeções do IDHM por Subprefeitura (2025-2035)")
    plt.xlabel("Ano")
    plt.ylabel("Projeção IDHM")
    plt.legend(loc="upper left", bbox_to_anchor=(1.05, 1), fontsize="small", ncol=2)
    plt.tight_layout()
    plt.savefig("analises/projecoes_idhm/projecoes_idhm.png", bbox_inches="tight", dpi=300)
    plt.close()

    print("Projeções concluídas com sucesso! Resultados salvos em analises/projecoes_idhm.")

except Exception as e:
    print(f"Ocorreu um erro durante a execução: {e}")

In [ ]:
# Criar previsões para 2020
for indicador in ["IDHM", "IDHM_Renda", "IDHM_Educacao"]:
    df[f"{indicador}_Linear_2020"] = df[f"{indicador}_2010"] + (df[f"{indicador}_2010"] - df[f"{indicador}_2000"])

    # Modelo Exponencial
    df[f"{indicador}_r"] = np.log(df[f"{indicador}_2010"] / df[f"{indicador}_2000"]) / 10
    df[f"{indicador}_Exp_2020"] = df[f"{indicador}_2000"] * np.exp(df[f"{indicador}_r"] * 20)

In [ ]:
# Salvar a saída em um novo arquivo
arquivo_saida = "idhm_projecao_2020.xlsx"
df.to_excel(arquivo_saida, index=False)

In [ ]:
print(f"Projeção concluída! Arquivo salvo como: {arquivo_saida}")